In [1]:
import time
import datetime
from mkndaq.inst.neph import NEPH
import schedule
import threading

cfg = {'ne300': {
                'type': 'NE300',
                'serial_number': '23-0690',
                'serial_id': 0,
                'protocol': 'acoem',
                'socket': {
                    'host': '192.168.3.150',
                    'port': 32783,
                    'timeout': 10,
                },
                'data_log': {
                    'parameters': [1000000, 2000000, 3000000, 6000000, 7000000, 8000000, 
                                   11000000, 12000000, 13000000, 14000000, 15000000, 16000000, 
                                   17000000, 18000000, 19000000, 20000000, 21000000, 26000000, 
                                   5001, 5002, 5003, 5004, 5005, 5006, 
                                   5010, 6007, 6008, 6001, 6002, 6003, 
                                   4035, 4036],
                    'wavelengths': [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
                    'angles': [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
                    'interval': 60, 
                },
                'get_data_interval': 2, # minutes. logger retrieval interval
                'zero_span_check_interval': 1500,
                'zero_check_duration': 30,
                'span_check_duration': 0,
                'staging_zip': True,  
                'verbosity': 2,  # 0: silent, 1: medium, 2: full          
            },
            'reporting_interval': 10,
            'logs': 'mkndaq/logs',
            'data': 'mkndaq/data',
            'staging': {
                'path': 'mkndaq/staging',
            },
}
ne300 = NEPH('ne300', cfg, verbosity=0)
# # Initialize NEPH (name: ne300  S/N: 23-0690)
#   Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.

# Initialize NEPH (name: ne300  S/N: 23-0690)
timed out


KeyboardInterrupt: 

In [ ]:
resp = ne300.get_values([5006])
time.sleep(.5)
print(resp)

In [ ]:
ne300.set_value(2030, 5006, verify=False, verbosity=2)
# ne300.set_value(4035, 1)

In [ ]:
ne300.get_data_log_config()

In [ ]:
import logging

def run_threaded(job_func):
    """Set up threading and start job.

    Args:
        job_func ([type]): [description]
    """
    job_thread = threading.Thread(target=job_func)
    job_thread.start()

# limit logging from schedule
logging.getLogger('schedule').setLevel(logging.CRITICAL)

# align start with a 10' timestamp
while int(time.time()) % 10 > 0:
    time.sleep(0.1)

fetch = 20
schedule.every(fetch).seconds.do(ne300.print_ssp_bssp)
schedule.every(cfg['ne300']['get_data_interval']).minutes.at(':10').do(run_threaded, ne300.get_new_data)
schedule.every(cfg['ne300']['zero_span_check_interval']).minutes.at(':00').do(run_threaded, ne300.do_zero_span_check)

while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
res = ne300.get_id()
print(res)

In [ ]:
# mimick VI099 response
ne300.get_current_data(strict=True, verbosity=0)

In [ ]:
ne300.get_current_data(verbosity=0)

In [ ]:
ne300.get_values([1635000, 1525000], verbosity=2)

In [ ]:
# retrieve VI099 command parameters
ne300.get_values(parameters=[1, 1635000,1525000,1450000,1635090,1525090,1450090,5001,5004,5003,5002,4036,4035])

In [ ]:
ne300.get_current_data(add_params=[1001, 1002, 1003, 1004, 1005, 2001, 2002])

In [ ]:
# not implemented
ne300.get_new_data()

In [ ]:
# not implemented
ne300.get_all_data()

In [ ]:
ne300.get_instr_type()

In [ ]:
ne300.get_datetime()

In [ ]:
ne300.get_data_log_config()

In [ ]:
# cycle through operating states
# 0: ambient, 1: zero, 2: span
for i in [1, 2, 0]:
    print(f"current: {ne300.get_current_operation()} > set: {i} ...")
    print(f"done: {ne300.set_current_operation(state=i)} okay")
    time.sleep(3)

In [ ]:
from datetime import datetime, timezone, timedelta
end = datetime.now(timezone.utc)
start = end - timedelta(minutes=10)
print(f"start: {start}\nend  : {end}")
data = ne300.get_logged_data(start=start, end=end, verbosity=1)
# for i in range(len(data)):
    # print(data[i])

In [ ]:
# repeat packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,1]))
response = ne300.tcpip_comm(message=message)
print(response)
display(ne300._acoem_decode_logged_data(response=response))

In [ ]:
# next packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,0]))
response = ne300.tcpip_comm(message=message)
response